# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [171]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests, json
# Import API key
from api_keys import geoapify_key

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [172]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,borskoye,53.0259,51.6987,266.22,90,99,5.27,RU,1675115204
1,1,jamestown,-15.9277,-5.7161,296.34,83,16,6.83,SH,1675115205
2,2,truro,50.2633,-5.0518,280.79,80,84,3.25,GB,1675115270
3,3,hobyo,5.3495,48.5264,296.97,66,98,9.71,SO,1675115271
4,4,chapleau,47.8456,-83.4019,258.18,47,40,3.60,CA,1675115271


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [177]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles= "OSM", 
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity",

)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [189]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_cities_df = city_data_df.loc[city_data_df["Humidity"] <= 35,:]

# Drop any rows with null values
narrow_cities_df = narrow_cities_df.dropna(axis=0, how="any") 

# Display sample data
narrow_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,ouadda,8.0716,22.3962,296.28,19,48,1.92,CF,1675115208
30,30,jalu,29.0331,21.5482,284.38,33,0,4.88,LY,1675115284
53,53,castro,45.8030,10.0671,276.53,10,2,2.82,IT,1675115299
71,71,jiexiu,37.0314,112.0017,274.59,14,24,1.29,CN,1675115310
94,94,arlit,18.7391,7.3933,291.14,4,100,4.11,NE,1675115324


### Step 3: Create a new DataFrame called `hotel_df`.

In [190]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrow_cities_df[["City","Country","Lat","Lng","Humidity"]].copy(deep=True)
hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
6,ouadda,CF,8.0716,22.3962,19,
30,jalu,LY,29.0331,21.5482,33,
53,castro,IT,45.8030,10.0671,10,
71,jiexiu,CN,37.0314,112.0017,14,
94,arlit,NE,18.7391,7.3933,4,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [191]:
import requests

# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    long_tude = row[3]
    lati_tude = row[2]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long_tude},{lati_tude},{radius}"
    params["bias"] = f"proximity:{long_tude},{lati_tude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(url,params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ouadda - nearest hotel: No hotel found
jalu - nearest hotel: فندق جالو السياحي
castro - nearest hotel: Continental
jiexiu - nearest hotel: 友谊宾馆
arlit - nearest hotel: No hotel found
santa rosalia - nearest hotel: No hotel found
mendoza - nearest hotel: Park Hyatt Mendoza
bereda - nearest hotel: No hotel found
redding - nearest hotel: Americana Modern Hotel
tezu - nearest hotel: No hotel found
coihaique - nearest hotel: Apart Hotel Tehuelche Natura
atar - nearest hotel: فندق سكليل
aswan - nearest hotel: El-Salam Hotel
kandi - nearest hotel: God Blessing
say - nearest hotel: No hotel found
rio gallegos - nearest hotel: Hospedaje Él Timón
xining - nearest hotel: 希岸酒店
amapa - nearest hotel: Hotel Santiago
portland - nearest hotel: Marriott
bilma - nearest hotel: No hotel found
goure - nearest hotel: No hotel found
umm jarr - nearest hotel: No hotel found
maghama - nearest hotel: No hotel found
san rafael - nearest hotel: Panama Hotel
jinchang - nearest hotel: No hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
6,ouadda,CF,8.0716,22.3962,19,No hotel found
30,jalu,LY,29.0331,21.5482,33,فندق جالو السياحي
53,castro,IT,45.8030,10.0671,10,Continental
71,jiexiu,CN,37.0314,112.0017,14,友谊宾馆
94,arlit,NE,18.7391,7.3933,4,No hotel found
113,santa rosalia,MX,26.6817,-105.4567,14,No hotel found
152,mendoza,AR,-32.8894,-68.8446,31,Park Hyatt Mendoza
163,bereda,NG,11.2135,8.3926,18,No hotel found
230,redding,US,40.5864,-122.3917,15,Americana Modern Hotel
258,tezu,IN,27.9254,96.1647,25,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [192]:
#%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles= "OSM", 
    frame_width = 800,
    frame_height = 600,
    color = "City",
    size = "Humidity",
    hover_cols="all"    

)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,index,Country,Hotel Name)